In [ ]:
!pip install gcloud

In [ ]:
!gcloud auth application-default login

In [3]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
##query de consumo ao BQ
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT ano, tipo_ocupacao,sigla_uf,id_municipio, sum(n_profissionais) as quantidade_vinculos  FROM `repositoriodedadosgpsp.Datalake.CNES_vinculo_saude_tipo_ocupacao_v2` where publico_privado='Público' group by ano, tipo_ocupacao, sigla_uf,id_municipio"
df = client.query(query).to_dataframe()

In [ ]:
#Removendo ocupações indesejadas
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()
df

In [7]:
df[df['tipo_ocupacao'].str.startswith('Médicos')]['tipo_ocupacao'].unique()
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.startswith('Médicos'),'Médicos',df['tipo_ocupacao'])
df
## Aqui eu unifico os três tipos de Médicos (clínica, cirurgia e terapeutico)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
0,2008,Profissionais da fisioterapia e afins,AP,1600600,2,Profissionais da fisioterapia e afins
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400159,46,Trabalhadores em serviços de promoção e apoio ...
2,2008,Farmacêuticos,RR,1400472,2,Farmacêuticos
3,2008,Técnicos de odontologia,DF,5300108,11,Técnicos de odontologia
4,2008,Técnicos e auxiliares de enfermagem,DF,5300108,3238,Técnicos e auxiliares de enfermagem
...,...,...,...,...,...,...
1836885,2024,Técnicos e auxiliares de enfermagem,TO,1712009,15,Técnicos e auxiliares de enfermagem
1836886,2024,Engenheiros civis e afins,TO,1721000,2,Engenheiros civis e afins
1836887,2024,Biomédicos,TO,1712157,1,Biomédicos
1836888,2024,Gestores e especialistas de operações em empre...,TO,1700301,2,Gestores e especialistas de operações em empre...


In [8]:
df.sort_values('quantidade_vinculos',ascending=False)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2
1701994,2024,Técnicos e auxiliares de enfermagem,SP,3550308,38738,Técnicos e auxiliares de enfermagem
1562059,2023,Técnicos e auxiliares de enfermagem,SP,3550308,36569,Técnicos e auxiliares de enfermagem
1421744,2022,Técnicos e auxiliares de enfermagem,SP,3550308,35973,Técnicos e auxiliares de enfermagem
224138,2011,Médicos,SP,3550308,35756,Médicos
66569,2010,Médicos,SP,3550308,35053,Médicos
...,...,...,...,...,...,...
1431077,2021,Fisioterapeutas,RS,4303202,1,Fisioterapeutas
1431078,2021,Operadores de equipamentos de entrada e transm...,RS,4318051,1,Operadores de equipamentos de entrada e transm...
1431079,2021,Fisioterapeutas,RS,4322541,1,Fisioterapeutas
1431080,2021,"Técnicos em secretariado, taquígrafos e esteno...",RS,4322202,1,"Técnicos em secretariado, taquígrafos e esteno..."


In [9]:
list(df.head(17)['tipo_ocupacao'])

['Profissionais da fisioterapia e afins',
 'Trabalhadores em serviços de promoção e apoio à saúde',
 'Farmacêuticos',
 'Técnicos de odontologia',
 'Técnicos e auxiliares de enfermagem',
 'Técnico em farmácia e em manipulação farmacêutica',
 'Trabalhadores em serviços de promoção e apoio à saúde',
 'Enfermeiros e afins',
 'Trabalhadores de laboratório fotográfico e radiológico',
 'Farmacêuticos',
 'Fonoaudiólogos',
 'Técnicos e auxiliares de enfermagem',
 'Técnicos e auxiliares de enfermagem',
 'Auxiliares de laboratório da saúde',
 'Trabalhadores em serviços de promoção e apoio à saúde',
 'Assistentes sociais e economistas domésticos',
 'Técnicos de laboratórios de saúde e bancos de sangue']

In [10]:
ocupacao_interesse = ['Técnicos e auxiliares de enfermagem',
 'Médicos',
 'Trabalhadores em serviços de promoção e apoio à saúde',
 'Enfermeiros e afins',
 'Cirurgiões-dentistas',
 'Fisioterapeutas',
 'Psicólogos e psicanalistas',
 'Farmacêuticos',
 'Técnicos de odontologia',
 'Tecnólogos e técnicos em métodos de diagnósticos e terapêutica',
 'Nutricionistas',
 'Assistentes sociais e economistas domésticos','Outros']

In [11]:
df['tipo_3'] = np.where(df['tipo_2'].isin(ocupacao_interesse) ,df['tipo_2'],'Outros')

In [12]:
df.head(20)

,ano,tipo_ocupacao,sigla_uf,id_municipio,quantidade_vinculos,tipo_2,tipo_3
0,2008,Profissionais da fisioterapia e afins,AP,1600600,2,Profissionais da fisioterapia e afins,Outros
1,2008,Trabalhadores em serviços de promoção e apoio ...,RR,1400159,46,Trabalhadores em serviços de promoção e apoio ...,Trabalhadores em serviços de promoção e apoio ...
2,2008,Farmacêuticos,RR,1400472,2,Farmacêuticos,Farmacêuticos
3,2008,Técnicos de odontologia,DF,5300108,11,Técnicos de odontologia,Técnicos de odontologia
4,2008,Técnicos e auxiliares de enfermagem,DF,5300108,3238,Técnicos e auxiliares de enfermagem,Técnicos e auxiliares de enfermagem
5,2008,Técnico em farmácia e em manipulação farmacêutica,RR,1400100,6,Técnico em farmácia e em manipulação farmacêutica,Outros
6,2008,Trabalhadores em serviços de promoção e apoio ...,AP,1600550,13,Trabalhadores em serviços de promoção e apoio ...,Trabalhadores em serviços de promoção e apoio ...
7,2008,Enfermeiros e afins,AP,1600709,1,Enfermeiros e afins,Enfermeiros e afins
8,2008,Trabalhadores de laboratório fotográfico e rad...,DF,5300108,1,Trabalhadores de laboratório fotográfico e rad...,Outros
9,2008,Farmacêuticos,RR,1400308,3,Farmacêuticos,Farmacêuticos


In [ ]:
x=df.pivot_table(index=['ano','sigla_uf'  ,'tipo_3'], values='quantidade_vinculos',aggfunc=np.sum)
y = pd.DataFrame(x)
y = y.reset_index()
##inserir a contagem como coluna

In [14]:
y

,ano,sigla_uf,tipo_3,quantidade_vinculos
0,2008,AC,Assistentes sociais e economistas domésticos,25
1,2008,AC,Cirurgiões-dentistas,110
2,2008,AC,Enfermeiros e afins,343
3,2008,AC,Farmacêuticos,78
4,2008,AC,Fisioterapeutas,46
...,...,...,...,...
5961,2024,TO,Psicólogos e psicanalistas,543
5962,2024,TO,Tecnólogos e técnicos em métodos de diagnóstic...,435
5963,2024,TO,Trabalhadores em serviços de promoção e apoio ...,5548
5964,2024,TO,Técnicos de odontologia,716


In [15]:
7966/2881900*1000

2.7641486519310177

In [16]:
## Ajeitando para o BQ

df= y.rename(columns={'tipo_3':'tipo_ocupacao'})

In [17]:
df.sort_values('quantidade_vinculos', ascending=False)

,ano,sigla_uf,tipo_ocupacao,quantidade_vinculos
5945,2024,SP,Médicos,161969
5594,2023,SP,Médicos,151851
5947,2024,SP,Outros,146277
5243,2022,SP,Médicos,143583
5596,2023,SP,Outros,138496
...,...,...,...,...
2729,2015,RR,Assistentes sociais e economistas domésticos,6
2379,2014,RR,Assistentes sociais e economistas domésticos,5
2385,2014,RR,Nutricionistas,5
2387,2014,RR,Psicólogos e psicanalistas,4


In [24]:
##Definindo intervalo dos dados
df = df[(df['ano'] > 2012) & (df['ano'] <= 2023)]
df

,ano,sigla_uf,tipo_ocupacao,quantidade_vinculos
1755,2013,AC,Assistentes sociais e economistas domésticos,60
1756,2013,AC,Cirurgiões-dentistas,264
1757,2013,AC,Enfermeiros e afins,747
1758,2013,AC,Farmacêuticos,128
1759,2013,AC,Fisioterapeutas,127
...,...,...,...,...
5610,2023,TO,Psicólogos e psicanalistas,525
5611,2023,TO,Tecnólogos e técnicos em métodos de diagnóstic...,439
5612,2023,TO,Trabalhadores em serviços de promoção e apoio ...,5473
5613,2023,TO,Técnicos de odontologia,676


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3860 entries, 1755 to 5614
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  3860 non-null   Int64 
 1   sigla_uf             3860 non-null   object
 2   tipo_ocupacao        3860 non-null   object
 3   quantidade_vinculos  3860 non-null   Int64 
dtypes: Int64(2), object(2)
memory usage: 287.4+ KB


In [28]:


schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos')
 ]


In [29]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [30]:
table_ref = dataset_ref.table('CNES_total_profissionais_saude_v2')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=a524d784-4700-49c6-b64a-034ed6a31de7>